In [43]:
import sys
sys.path.insert(0, '../scripts/')
from helper_functions import convert_census_to_postcode

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from statsmodels.tsa.vector_ar.var_model import VAR

import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols

import geopandas as gpd

from scipy.interpolate import interp1d

output_dir = '../data/curated/'


<h3> Preparation for forecasting </h3>

In [4]:
properties_df = pd.read_csv("../data/curated/properties_processed.csv", index_col=0)
properties_df['Postcode'] = properties_df['Postcode'].astype(str)

# read in census dataframes
census_df = pd.read_csv("../data/curated/census_data.csv")
sa2_postcode_map = pd.read_csv("../data/curated/sa2_postcode_mapping_2021.csv")

In [5]:
census_df_postcode_agg = convert_census_to_postcode(census_df, sa2_postcode_map, 'mean_no_zero')

Median Rental Prices

In [7]:
median_rent = properties_df.groupby(["Postcode"])["Cost"].median()


In [8]:
median_rent.to_csv(f'{output_dir}median_rental_postcode.csv')

In [78]:
impute_features = ['tot_population', 'avg_med_mortg_rep', 'avg_med_person_inc', 'avg_med_rent', 'avg_med_hh_inc', 'tot_avg_hh_size']
init_years = [11, 16, 21]
new_years = list(range(11, 22))


for i in [2]:
    data = census_df_postcode_agg.iloc[i]
    #print(census_df_postcode_agg.iloc[i])

    # interpolation 
    interpolated_dict = dict()
    interpolated_dict['date']= [f'20{year}' for year in new_years]

    for feature in impute_features:
        init_y = data[[f'{feature}_{init_years[0]}', f'{feature}_{init_years[1]}', f'{feature}_{init_years[2]}']].tolist()
        #print(init_y)

        interp_func = interp1d(init_years, init_y)
        new_y = list(interp_func(new_years))
        #print(new_y)

        interpolated_dict[feature] = new_y

    # creation of new dataframe
    interpolated_df = pd.DataFrame(interpolated_dict)
    

    # model
    interpolated_df['date'] = pd.to_datetime(interpolated_df['date'])
    interpolated_df = interpolated_df.set_index('date')

    model = VAR(endog=interpolated_df)
    model_fit = model.fit()

    forecasted_values = model_fit.forecast(interpolated_df.values[-1:], 5)

    forecasted_avg_med_rent = [row[3] for row in forecasted_values]
    forecasted_years = range(2022, 2022+len(forecasted_avg_med_rent))
    print(list(forecasted_years))

    break 

[2022, 2023, 2024, 2025, 2026]


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency AS-JAN will be used.
  self._init_dates(dates, freq)


In [65]:
interpolated_df

,tot_population,avg_med_mortg_rep,avg_med_person_inc,avg_med_rent,avg_med_hh_inc,tot_avg_hh_size
date,,,,,,
2011-01-01,15496.0,2200.0,701.5,418.5,1493.5,2.15
2012-01-01,16523.4,2170.0,704.4,413.8,1488.0,2.15
2013-01-01,17550.8,2140.0,707.3,409.1,1482.5,2.15
2014-01-01,18578.2,2110.0,710.2,404.4,1477.0,2.15
2015-01-01,19605.6,2080.0,713.1,399.7,1471.5,2.15
2016-01-01,20633.0,2050.0,716.0,395.0,1466.0,2.15
2017-01-01,21123.0,2057.0,772.8,393.1,1523.0,2.11
2018-01-01,21613.0,2064.0,829.6,391.2,1580.0,2.07
2019-01-01,22103.0,2071.0,886.4,389.3,1637.0,2.03
